# TESLA Stock Price Prediciton using Historical Stock Data and Sentiment Analysis

## Project Overview

To-Do:
Brief introduction to project, objectives and scope 
Describe problem statement, predicting stock prices, sentiment analysis and dataset used.

Also descriptions underneath each subtitle

## Data Acquisition and Preprocessing
To-Do: Describe sources from which you optain your data (yfinance and Twitter API), detail steps taken to collect and preprocess the data, data cleaning and handling missing values. Justify using only 2020 onwards.

Explain some stock data will have irregular trading days like holidays and stock splits and dividends can artifically change value  before returnign to normal(explain), sorted by yfinance and auto_adjust

- Data cleaning for future datasets (missing values, data type conversion, removing duplicates)

### Importing Libraries

In [112]:
import yfinance as yf
import pandas as pd
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from sklearn.preprocessing import MinMaxScaler

### Acquiring Data

In [113]:
# Set display options to show four decimal places
pd.set_option('display.float_format', '{:.4f}'.format)

# Download Tesla data using yfinance
ticker = 'TSLA'
# Auto_adjust set to true to account for stock splits and dividends
data = yf.download(ticker, auto_adjust=True)

# Convert downloaded data into a Pandas DataFrame
tsla_df = pd.DataFrame(data, columns=['Open', 'High', 'Low', 'Close', 'Volume'])

# Start date for Tesla stock prices
start_date = '2020-01-01'

# End date for Tesla stock prices
end_date = pd.Timestamp(datetime.date.today())
                          
# End date for Tesla stock prices
# Select data from 01/01/2020 onwards
tsla_df = tsla_df.loc[start_date : end_date]

# Columns indicating price
price_columns = ['Open', 'High', 'Low', 'Close']

# Plot colours for Open, High, Low and Close respectively
price_colours = ['#828cf8', '#ef553b', '#00cc96', '#b983f8']

# Colour for volume plots
volume_colour = '#ffa500'

[*********************100%%**********************]  1 of 1 completed


In [114]:
tsla_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2020-01-02,28.3000,28.7133,28.1140,28.6840,142981500
2020-01-03,29.3667,30.2667,29.1280,29.5340,266677500
2020-01-06,29.3647,30.1040,29.3333,30.1027,151995000
2020-01-07,30.7600,31.4420,30.2240,31.2707,268231500
2020-01-08,31.5800,33.2327,31.2153,32.8093,467164500


## Exploratory Data Analysis

### Key Statistics

In [115]:
tsla_df.describe()

,Open,High,Low,Close,Volume
count,1049.0000,1049.0000,1049.0000,1049.0000,1049.0000
mean,209.0136,213.7072,203.9286,208.9290,132123815.5386
std,84.3024,85.9604,82.3409,84.1096,87047601.5785
min,24.9800,26.9907,23.3673,24.0813,29401800.0000
25%,164.0000,167.4967,159.9100,163.2033,78855600.0000
50%,222.5300,226.6533,216.6667,221.3100,106827000.0000
75%,260.3000,266.3333,256.4367,260.5400,153391400.0000
max,411.4700,414.4967,405.6667,409.9700,914082000.0000


In [116]:
# Create a list of Box traces for Price columns 
price_box_traces = []

for i, column in enumerate(price_columns):
    price_box_traces.append(go.Box(y=tsla_df[column],
                                   name=column,
                                  marker=dict(color=price_colours[i])))

# Create the figure and add the traces
fig = go.Figure(data=price_box_traces)

# Update the layout
fig.update_layout(
    title='Box Plot of Tesla Stock Prices',
    yaxis_title='Value ($)'
)

# Show the plot
fig.show()

In [117]:
# Create Box trace for Volume
volume_box_trace = go.Box(y=tsla_df['Volume'],
                          name='Volume',
                         marker=dict(color=volume_colour))

# Create the figure and add the trace
fig = go.Figure(data=volume_box_trace)

# Update the layout
fig.update_layout(
    title='Box Plot of Tesla Volume',
    yaxis_title='Volume'
)

# Show the plot
fig.show()

GIVE A DESCRIPTION OF THE ABOVE VISUALISATION (volume generally simial rbut large outliers probably due to news events) compare to price changes

### Distributions

In [118]:
# Fetch data and labels for Price columns
price_data = [tsla_df[column] for column in price_columns]
group_labels = price_columns

# Create the figure and add data
fig = ff.create_distplot(price_data,
                         group_labels,
                         show_hist=False,
                         show_rug=False,
                         colors=price_colours
                        )

# Update the layout
fig.update_layout(
    title='Distribution of Tesla Stock Prices',
    xaxis_title='Value ($)'
    )

# Show the figure
fig.show()

DESCRIBE VALUE WITH DENSITY PLOT

In [119]:
# Fetch data and labels for Volume column
volume_data = [tsla_df['Volume']]
group_labels = ['Volume']

# Create the figure and add data
fig = ff.create_distplot(volume_data,
                         group_labels,
                         show_hist=False,
                         show_rug=False,
                         colors=[volume_colour]
                        )

# Update the layout
fig.update_layout(
    title='Distribution of Tesla Volume',
    xaxis_title='Volume'
    )

# Show the figure
fig.show()

EXPLAIN VISUALISATIONS

### Trends over Time

CHANGE TO SMA
EXPLAIN EMA AND EMA PARAMETERS AND ROLLING STANDARD DEVIATION AND EMA VS SMA BASED ON CLOSE

In [120]:
# Window size of 25 chosen for smooth but responsive EMA curve
window = 25

# Calculate the Exponential Weighted Moving Average
tsla_df['EMA'] = tsla_df['Close'].ewm(span=window, min_periods=window, adjust=False).mean()

# Calculate the rolling Standard Deviation
tsla_df['STD'] = tsla_df['Close'].rolling(window=window).std()

# Create the traces for the Candlestick Plot
candlestick_trace = go.Candlestick(x=tsla_df.index,
                                   open=tsla_df['Open'],
                                   high=tsla_df['High'],
                                   low=tsla_df['Low'],
                                   close=tsla_df['Close'],
                                   name='Candlestick')

# Create the trace for the EMA Line Plot
ema_trace = go.Scatter(x=tsla_df.index,
                       y=tsla_df['EMA'],
                       mode='lines',
                       name='EMA',
                       line=dict(color='blue'))

# Create the trace for the Standard Deviation Line Plot
std_trace = go.Scatter(x=tsla_df.index,
                       y=tsla_df['STD'],
                       mode='lines',
                       name='Standard Deviation',
                       line=dict(color='purple'))

# Create subplots to plot Standard Deviation on the same x-axis but 
# a different plot to the Candlestick Chart
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

# Add the relevant traces to the correct subplot
fig.add_trace(candlestick_trace, row=1, col=1)

fig.add_trace(ema_trace, row=1, col=1)

fig.add_trace(std_trace, row=2, col=1)

# Update the layout
fig.update_yaxes(title_text="Price", row=1, col=1)

fig.update_yaxes(title_text="STD", row=2, col=1, secondary_y=True)

fig.update_layout(title='Candlestick Chart of Tesla Stock with EMA and Rolling Standard Deviation',
                  xaxis_rangeslider_visible=False)

# Show the figure
fig.show()


EXPLAIN VISUALISATION

In [121]:
# Window size of 25 chosen for smooth but responsive SMA curve
window = 25

# Calculate the Simple Moving Average
tsla_df['Volume_SMA'] = tsla_df['Volume'].rolling(window=window).mean()

# Calculate the rolling Standard Deviation
tsla_df['Volume_STD'] = tsla_df['Volume'].rolling(window=window).std()

# Create the trace for the Volume Line Plot
volume_trace = go.Scatter(x=tsla_df.index,
                       y=tsla_df['Volume'],
                       mode='lines',
                       name='Volume',
                       line=dict(color=volume_colour))

# Create the trace for the Volume SMA Line Plot
volume_sma_trace = go.Scatter(x=tsla_df.index,
                            y=tsla_df['Volume_SMA'],
                            mode='lines',
                            name='Simple Moving Average',
                            line=dict(color='blue'))

# Create the trace for the Volume Standard Deviation Line Plot
volume_std_trace = go.Scatter(x=tsla_df.index,
                             y=tsla_df['Volume_STD'],
                             mode='lines',
                             name='Standard Deviation',
                             line=dict(color='purple'))

# Create subplots to plot Standard Deviation on the same x-axis but 
# a different plot to the Volume Line Plot
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

# Add the relevant traces to the correct subplot
fig.add_trace(volume_trace, row=1, col=1)

fig.add_trace(volume_sma_trace, row=1, col=1)

fig.add_trace(volume_std_trace, row=2, col=1)

# Update the layout
fig.update_yaxes(title_text="Price", row=1, col=1)

fig.update_yaxes(title_text="STD", row=2, col=1, secondary_y=True)

fig.update_layout(title='Tesla Volume Over Time Plot with Simple Moving Average and Rolling Standard Deviation',
                  xaxis_rangeslider_visible=False)

# Show the figure
fig.show()

EXPLAIN VISUALISATIONS

### Correlations among Variables

In [122]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Apply Min-Max Scaling to the DataFrame (excluding any non-numeric columns)
tsla_df_normalized = tsla_df[['Open', 'High', 'Low', 'Close', 'Volume']].copy() # Make a copy to avoid modifying the original DataFrame
tsla_df_normalized[tsla_df_normalized.columns] = scaler.fit_transform(tsla_df_normalized[tsla_df_normalized.columns])
# Now tsla_df_normalized contains normalized values between 0 and 1
tsla_df_normalized

,Open,High,Low,Close,Volume
Date,,,,,
2020-01-02,0.0086,0.0044,0.0124,0.0119,0.1284
2020-01-03,0.0114,0.0085,0.0151,0.0141,0.2682
2020-01-06,0.0113,0.0080,0.0156,0.0156,0.1386
2020-01-07,0.0150,0.0115,0.0179,0.0186,0.2700
2020-01-08,0.0171,0.0161,0.0205,0.0226,0.4948
...,...,...,...,...,...
2024-02-27,0.4633,0.4609,0.4575,0.4552,0.0896
2024-02-28,0.4539,0.4601,0.4579,0.4612,0.0796
2024-02-29,0.4637,0.4601,0.4580,0.4608,0.0639


In [123]:
# Define the correlation matrix
correlation_matrix = tsla_df_normalized[['High', 'Low', 'Open', 'Close', 'Volume']].corr()

correlation_matrix

,High,Low,Open,Close,Volume
High,1.0000,0.9983,0.9987,0.9983,-0.6302
Low,0.9983,1.0000,0.9982,0.9985,-0.6496
Open,0.9987,0.9982,1.0000,0.9963,-0.6398
Close,0.9983,0.9985,0.9963,1.0000,-0.6381
Volume,-0.6302,-0.6496,-0.6398,-0.6381,1.0000
